In [1]:
import numpy as np
from szar.counts import ClusterCosmology,Halo_MF
from szar.szproperties import SZ_Cluster_Model
from configparser import SafeConfigParser
from orphics.tools.io import dictFromSection,listFromConfig
import cPickle as pickle
import time

home = '/Users/nab/Repos/SZ_filter/'

!pwd

iniFile = home+'input/pipeline.ini'
expName = 'CCATP-propv2'
gridName = 'grid-owl2'
cal = 'owl2'

Config = SafeConfigParser()
Config.optionxform=str
Config.read(iniFile)

fparams = {}
for (key, val) in Config.items('params'):
    if ',' in val:
        param, step = val.split(',')
        fparams[key] = float(param)
    else:
        fparams[key] = float(val)

bigDataDir = Config.get('general','bigDataDirectory')
clttfile = Config.get('general','clttfile')
constDict = dictFromSection(Config,'constants')
clusterDict = dictFromSection(Config,'cluster_params')
beam = listFromConfig(Config,expName,'beams')
noise = listFromConfig(Config,expName,'noises')
freq = listFromConfig(Config,expName,'freqs')
lknee = listFromConfig(Config,expName,'lknee')[0]
alpha = listFromConfig(Config,expName,'alpha')[0]

calFile = bigDataDir+"lensgrid_grid-"+cal+"_"+cal+".pkl"

version = Config.get('general','version')

mexp_edges, z_edges, lndM = pickle.load(open(calFile,"rb"))

mgrid,zgrid,siggrid = pickle.load(open(bigDataDir+"szgrid_"+expName+"_"+gridName+ "_v" + version+".pkl",'rb'))

assert np.all(mgrid==mexp_edges)
assert np.all(z_edges==zgrid)

#print lndM

cc = ClusterCosmology(fparams,constDict,clTTFixFile=clttfile)
HMF = Halo_MF(cc,mgrid,zgrid)

qs = listFromConfig(Config,'general','qbins')
qspacing = Config.get('general','qbins_spacing')
if qspacing=="log":
    qbin_edges = np.logspace(np.log10(qs[0]),np.log10(qs[1]),int(qs[2])+1)
elif qspacing=="linear":
    qbin_edges = np.linspace(qs[0],qs[1],int(qs[2])+1)
else:
    raise ValueError
    
SZProp = SZ_Cluster_Model(cc,clusterDict,rms_noises = noise,fwhms=beam,freqs=freq,lknee=lknee,alpha=alpha)

Mwl = 10**HMF.Mexp
z_arr = HMF.zarr
M_arr =  np.outer(HMF.M,np.ones([len(z_arr)]))

#if HMF.sigN is None: HMF.updateSigN(SZProp)
#sigN = HMF.sigN
#np.savetxt(home+'/tests/sigN_arr_save.txt',sigN)

sigN = np.loadtxt(home+'/tests/sigN_arr_save.txt')

#SZProp = SZ_Cluster_Model(cc,clusterDict,rms_noises = noise,fwhms=beam,freqs=freq,lknee=lknee,alpha=alpha)

q_arr = (qbin_edges[1:]+qbin_edges[:-1])/2.

i = 5
kk = 2
jj = 10

start = time.time()
blah_orig = SZProp.P_of_qn(SZProp.lnY,M_arr[:,i],z_arr[i],sigN[:,i],q_arr[kk])
print time.time() - start
blah_mwl = SZProp.Mwl_prob (Mwl[jj],M_arr[:,i],lndM[:,i])

print z_arr.size,q_arr.size,HMF.Mexp.size,Mwl.size

start = time.time()
blah = SZProp.P_of_qn_corr(SZProp.lnY,M_arr[:,i],z_arr[i],sigN[:,i],q_arr[kk],Mwl[jj],lndM[:,i])
print time.time() - start

start = time.time()
SZProp.Pfunc_qarr(sigN,HMF.M,z_arr,q_arr)
print time.time() - start
#start = time.time()
#SZProp.Pfunc_qarr_corr(sigN,HMF.M,z_arr,q_arr,HMF.Mexp,lndM)
#print time.time() - start

#print blah_orig * blah_mwl
#print blah

#dN_dmqz_corr = HMF.N_of_mqz_SZ_corr(lndM,qbin_edges,SZProp)                                                                  
#dN_dmqz = HMF.N_of_mqz_SZ(lndM,qbin_edges,SZProp)

/Users/nab/Repos/orphics/orphics/tools/io.py:3: UserWarning: WARNING: This module is deprecated. Most of its contents have moved to orphics.io. If you do not find the function you require there, please raise an issue.
  warnings.warn("WARNING: This module is deprecated. Most of its contents have moved to orphics.io. If you do not find the function you require there, please raise an issue.")
/Users/nab/Repos/orphics/orphics/tools/stats.py:3: UserWarning: WARNING: This module is deprecated. Most of its contents have moved to orphics.stats. If you do not find the function you require there, please raise an issue.
  warnings.warn("WARNING: This module is deprecated. Most of its contents have moved to orphics.stats. If you do not find the function you require there, please raise an issue.")
/Users/nab/Repos/orphics/orphics/analysis/flatMaps.py:3: UserWarning: WARNING: This module is deprecated. Most of its contents have moved to orphics.maps. If you do not find the function you require ther

/Users/nab/Repos/SZ_filter
0.00953698158264
20 64 222 222
0.0182118415833
9.79920601845


/Users/nab/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:18: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.
/Users/nab/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.


In [2]:


print (blah - (blah_orig * blah_mwl)) / blah

[  2.27139778e-16  -6.29390912e-16   3.86754846e-16  -8.88983090e-16
  -1.63001348e-16  -7.44803974e-16   2.71319117e-16  -6.15460829e-16
   5.56293601e-16   0.00000000e+00  -3.59208816e-16  -8.01801377e-16
   2.85162295e-16   2.52495577e-16   6.67980513e-16  -7.82293920e-16
   6.84524705e-16  -4.47626726e-16   5.18659443e-16  -4.49371832e-16
  -3.88301462e-16   0.00000000e+00   1.43808292e-16   1.23213832e-16
   4.20861252e-16  -5.37113839e-16   6.06914709e-16   0.00000000e+00
   0.00000000e+00  -8.99270447e-16  -4.48842177e-16   3.71499322e-16
  -1.01996770e-15  -5.01718689e-16   4.09500274e-16   2.21888757e-16
   3.59276685e-16  -5.79600306e-16  -2.32961517e-16  -1.86674196e-16
   0.00000000e+00  -3.56399271e-16   1.88749572e-16  -1.49557215e-16
  -3.54681772e-16  -3.73094948e-16  -1.46942288e-16  -1.15625416e-16
  -1.81899348e-16   0.00000000e+00   1.12820852e-16  -1.78296960e-16
  -2.82780878e-16  -2.25277839e-16  -3.60952738e-16   2.91079764e-16
  -2.36496426e-16   1.93739003e-16